## Precompute metrics for use in other notebooks and for aggregate metrics API

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2023
month = 10
program = "default"
dynamic_config = {}
study_type = "study"
include_test_users = False
labels = {}
use_imperial = True
sensed_algo_prefix = "cleaned"
survey_info = {}

In [ ]:
import scaffolding
import arrow
import pymongo

import emission.core.get_database as edb
import emission.storage.timeseries.fmt_time_query as estf
import emission.storage.decorations.analysis_timeseries_queries as esda
import emcommon.metrics.metrics_summaries as emcms

In [ ]:
if year is None or month is None:
    ipython = get_ipython()
    ipython._showtraceback = scaffolding.no_traceback_handler
    raise Exception("This notebook only runs when year and month are defined.")

In [ ]:
DEFAULT_METRIC_LIST = {
    'footprint': ['mode_confirm'],
    'distance': ['mode_confirm'],
    'duration': ['mode_confirm'],
    'count': ['mode_confirm'],
}

metric_list = dynamic_config.get('metrics', {}).get('phone_dashboard_ui', {}).get('metric_list', DEFAULT_METRIC_LIST)

if 'label_options' not in dynamic_config:
    DEFAULT_LABEL_OPTIONS = await emcu.read_json_resource("label-options.default.json")
    dynamic_config['label_options'] = DEFAULT_LABEL_OPTIONS

In [ ]:
tq = scaffolding.get_time_query(year, month)
ct = esda.get_entries(esda.CONFIRMED_TRIP_KEY, None, tq)

metrics = await emcms.generate_summaries(metric_list, ct, dynamic_config)

agg_metrics_db = edb.get_agg_metrics_db()
op_list = []
skipped_count, upserted_count, modified_count = 0, 0, 0

for metric_name, days in metrics.items():
    for day in days:
        date_str = day["date"]
        n_users = day["nUsers"]
        data_fields = {
            k: v
            for k, v in day.items()
            if k not in ("date", "nUsers")
        }
        query = {"date": date_str, "metric": metric_name}

        existing = agg_metrics_db.find_one(query)
        needs_update = (
            not existing or
            existing.get("nUsers") != n_users or
            existing.get("data") != data_fields
        )

        if needs_update:
            doc = {
                "date": date_str,
                "metric": metric_name,
                "nUsers": n_users,
                "data": data_fields,
                "last_updated": arrow.utcnow().datetime
            }
            op_list.append(pymongo.ReplaceOne(query, doc, upsert=True))
        else:
            skipped_count += 1

if op_list:
    result = agg_metrics_db.bulk_write(op_list)
    upserted_count = result.upserted_count
    modified_count = result.modified_count

print(f"Upserted {upserted_count}, modified {modified_count}, skipped {skipped_count}")